In [0]:
dbutils.widgets.text("p_file_date","2024-12-23")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
#movies_df = spark.sql("select * from movie_silver.movies").filter(f"file_date = '{v_file_date}'")
movies_df = spark.read.format("delta").load(f"{silver_folder_path}/movies") \
                    .filter(f"file_date = '{v_file_date}'")


In [0]:
#languages_df = spark.sql("select * from movie_silver.languages")
languages_df = spark.read.format("delta").load(f"{silver_folder_path}/languages")
                    

In [0]:
#movies_languages_df = spark.sql("select * from movie_silver.movie_language").filter(f"file_date = '{v_file_date}'")
movies_languages_df = spark.read.format("delta").load(f"{silver_folder_path}/movie_language") \
                    .filter(f"file_date = '{v_file_date}'")

In [0]:
#genres_df = spark.sql("select * from movie_silver.genres")
genres_df = spark.read.format("delta").load(f"{silver_folder_path}/genres")
                    

In [0]:
#movies_genres_df = spark.sql("select * from movie_silver.movie_genre").filter(f"file_date = '{v_file_date}'")
movies_genres_df = spark.read.format("delta").load(f"{silver_folder_path}/movie_genre") \
                    .filter(f"file_date = '{v_file_date}'")

In [0]:
languages_mov_lan_df = languages_df.join(movies_languages_df, 
                                          languages_df.language_id == movies_languages_df.lenguage_id,
                                          "inner") \
                                    .select(languages_df.language_id, languages_df.language_name, movies_languages_df.movie_id)

        

In [0]:
genres_mov_gen_df = genres_df.join(movies_genres_df, 
                                   genres_df.genre_id == movies_genres_df.genre_id,
                                   "inner") \
                                .select(genres_df.genre_name,genres_df.genre_id , movies_genres_df.movie_id)


In [0]:
movie_filter_df = movies_df.filter("year_release_date >= 2000")

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
results_movies_genres_languages_df = movie_filter_df.join(languages_mov_lan_df, 
                                                          movie_filter_df.movie_id == languages_mov_lan_df.movie_id,
                                                          "inner") \
                                    .join(genres_mov_gen_df, 
                                          movie_filter_df.movie_id == genres_mov_gen_df.movie_id,
                                        "inner")

In [0]:
result_df = results_movies_genres_languages_df \
                    .select(movie_filter_df.movie_id.alias('movie_id'),'language_id','genre_id', 'title', 'duration_time', 'release_date', 'vote_average', 'language_name', 'genre_name') \
                    .withColumn("created_date", lit(v_file_date)) \
                    

                    
                    

#### Escribir datos en datalake en formato "Parquet"

In [0]:
#overwrite_partition(result_df, "movie_gold", "results_movie_genre_language", "created_date") 

In [0]:
#result_df.write.mode("append").partitionBy("created_date").format("parquet").saveAsTable("movie_gold.results_movie_genre_language")

#Escribir en formato 'Delta'

In [0]:
merge_key = 'tgt.movie_id = src.movie_id and tgt.language_id = src.language_id and tgt.genre_id = src.genre_id'
merge_delta_data(result_df,"/mnt/moviehistory7809/gold","movie_gold", "results_movie_genre_language",  "created_date", merge_key)

In [0]:
%sql
select created_date,count(*) 
from movie_gold.results_movie_genre_language
group by created_date